# EDA

In this dataset, we are presented with sales data. Following columns are available:

- date
- country
- store
- product
- num_sold - quantity of product sold in given day

The task is to use this data to predict sales for the next year.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data_path = "./data/"
df_train = pd.read_csv(data_path + "train.csv")
df_test = pd.read_csv(data_path + "test.csv")

# drop id
df_train.drop('id', axis=1, inplace=True)

df_train['date'] = pd.to_datetime(df_train['date'])

df_train.head(10)

In [ ]:
# check if there are any null values
df_train.isnull().sum()

### date column

In [ ]:
start_date = df_train['date'].min()
end_date = df_train['date'].max()
number_of_years = (end_date - start_date).days / 365
print("Start date: ", start_date)
print("End date: ", end_date)
print("Number of years: ", number_of_years)

### Total sales

In [ ]:
# group by date and sum the num_sold as total_sales

total_sales_daily = df_train[['date', 'num_sold']].groupby('date').sum().reset_index()
total_sales_weekly = df_train[['date', 'num_sold']].groupby('date').sum().resample('W').sum().reset_index()

total_sales_daily.columns = ['date', 'total_sales']
total_sales_weekly.columns = ['date', 'total_sales']

total_sales_daily

In [ ]:
def plot_time_series_data(df, title='Total Sales Daily', color='dodgerblue'):
    fig, ax = plt.subplots(figsize=(15, 5))
    ax.plot(df['date'], df['total_sales'], color=color)
    ax.set_xlabel('Date')
    ax.set_ylabel('Total Sales')
    ax.set_title(title)
    plt.show()

plot_time_series_data(total_sales_daily, title='Total Sales Daily')

In [ ]:

plot_time_series_data(total_sales_weekly, title='Total Sales Weekly', color='lightcoral')

In [ ]:
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'] # The day of the week with Monday=0, Sunday=6. - pandas doc

def boxplot_sales_by_column(df, column, order=None, color_palette='Set1'):
    fig, ax = plt.subplots(figsize=(15, 5))
    sns.boxplot(x=column, y='num_sold', data=df, ax=ax, order=order, palette=color_palette)
    ax.set_ylabel('Number of Sales')
    ax.set_xlabel('')
    ax.set_title(f'Number of Sales by {column}')
    plt.show()

df_train['weekday'] = df_train['date'].dt.weekday
# translate weekday to weekday name
df_train['weekday'] = df_train['weekday'].map(lambda x: days_of_week[x])
boxplot_sales_by_column(df_train, 'weekday', order=days_of_week)

In [ ]:
def barplot_sales_by_column(df, column, order=None, color_palette='Set1'):
    fig, ax = plt.subplots(figsize=(15, 5))
    sns.barplot(x=column, y='num_sold', data=df, ax=ax, order=order, palette=color_palette)
    ax.set_ylabel('Number of Sales')
    ax.set_xlabel('')
    ax.set_title(f'Number of Sales by {column}')
    plt.show()

barplot_sales_by_column(df_train, 'weekday', order=days_of_week)

Observations: 

- As described on competition page, dataset contains yearly seasonality (peaks at the beginning of each year)
- Data is inspired by real world as we can see dip in sales in April 2020
- Biggest sales are happening Sunday, Saturday and Friday, from the 1st to the 3rd biggest

### Countries

In [ ]:
print("Number of unique countries: ", df_train['country'].nunique())
total_sales_by_country = df_train[['country', 'num_sold']].groupby('country').sum().reset_index().sort_values(by='num_sold', ascending=False)
total_sales_by_country

In [ ]:
# assign each country to a color
country_color_palette = dict(zip(total_sales_by_country['country'], sns.color_palette('Set1', len(total_sales_by_country['country']))))

In [ ]:
boxplot_sales_by_column(df_train, 'country', order=total_sales_by_country['country'], color_palette=country_color_palette)

In [ ]:
barplot_sales_by_column(df_train, 'country', order=total_sales_by_country['country'], color_palette=country_color_palette)

In [ ]:
total_sales_by_country_daily = df_train[['date', 'country', 'num_sold']].groupby(['date', 'country']).sum().reset_index()
total_sales_by_country_daily.columns = ['date', 'country', 'total_sales']
total_sales_by_country_daily

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

sns.lineplot(x='date', y='total_sales', hue='country', data=total_sales_by_country_daily, ax=ax, palette=country_color_palette)
ax.set_xlabel('Date')
ax.set_title('Number of Sales by Country')

Observations: 

- Canada is the biggest country in sales
- Each country follows similar pattern, with two exceptions: Japan does not have its peaks at the beginning of each year, but during the first half. Canada at the beginning of 2021 has a more significant jump in sales compared to other countries.
- Spain during peaks at the beginning of each year surpasses sales in Japan (which, again, has no peaks there)

### store



In [ ]:
print("Number of unique stores: ", df_train['store'].nunique())
total_sales_by_store = df_train[['store', 'num_sold']].groupby('store').sum().reset_index().sort_values(by='num_sold', ascending=False)
total_sales_by_store

In [ ]:
total_sales_by_store_daily = df_train[['date', 'store', 'num_sold']].groupby(['date', 'store']).sum().reset_index()
total_sales_by_store_daily.columns = ['date', 'store', 'total_sales']

fig, ax = plt.subplots(figsize=(15, 5))

sns.lineplot(x='date', y='total_sales', hue='store', data=total_sales_by_store_daily, ax=ax, palette='Set1')

# for each store plot the average number of sales between 2017-01-01 and 2020-01-01
start_date = '2017-01-01'
end_date = '2020-01-01'

total_sales_by_store_daily_2017_2020 = total_sales_by_store_daily[(total_sales_by_store_daily['date'] >= start_date) & (total_sales_by_store_daily['date'] < end_date)]

# for each store calculate its average number of sales
average_sales_by_store = total_sales_by_store_daily_2017_2020[['store', 'total_sales']].groupby('store').mean().reset_index()
average_sales_by_store.columns = ['store', 'average_sales']

# plot the average number of sales for each store as horizontal line on existing plot
for store in average_sales_by_store['store']:
    average_sales = average_sales_by_store[average_sales_by_store['store'] == store]['average_sales'].values[0]
    ax.axhline(y=average_sales, color='black', linestyle='--')

ax.set_xlabel('Date')
ax.set_title('Number of Sales by store')
average_sales_by_store

In [ ]:
# "Kagglazon"

lowest_sales_by_store_kagglazon = df_train[df_train['store'] == 'Kagglazon'][['date', 'num_sold']].groupby('date').sum().reset_index().sort_values(by='num_sold', ascending=True)
lowest_sales_by_store_kagglazon.head(3)

# take lowest value
lowest_sales_kagglazon = lowest_sales_by_store_kagglazon['num_sold'].min()
lowest_sales_kagglazon

# compare the percentage drop in sales using average_sales_by_store
average_sales_kagglazon = average_sales_by_store[average_sales_by_store['store'] == 'Kagglazon']['average_sales'].values[0]

percentage_drop_kagglazon = (lowest_sales_kagglazon - average_sales_kagglazon) / average_sales_kagglazon * 100
"Between the end of 2019 and april 2020, Kagglazon experienced a drop of {:.2f}% in sales".format(percentage_drop_kagglazon)

In [ ]:
# "Kaggle Learn"

lowest_sales_by_store_learn = df_train[df_train['store'] == 'Kaggle Learn'][['date', 'num_sold']].groupby('date').sum().reset_index().sort_values(by='num_sold', ascending=True)
lowest_sales_by_store_learn.head(3)

# take lowest value
lowest_sales_learn = lowest_sales_by_store_learn['num_sold'].min()

# compare the percentage drop in sales using average_sales_by_store
average_sales_learn = average_sales_by_store[average_sales_by_store['store'] == 'Kaggle Learn']['average_sales'].values[0]

percentage_drop_learn = (lowest_sales_learn - average_sales_learn) / average_sales_learn * 100
"Between the end of 2019 and april 2020, Kaggle Learn experienced a drop of {:.2f}% in sales".format(percentage_drop_learn)

In [ ]:
#  "Kaggle Store"

lowest_sales_by_store_store = df_train[df_train['store'] == 'Kaggle Store'][['date', 'num_sold']].groupby('date').sum().reset_index().sort_values(by='num_sold', ascending=True)
lowest_sales_by_store_store.head(3)

# take lowest value
lowest_sales_store = lowest_sales_by_store_store['num_sold'].min()

# compare the percentage drop in sales using average_sales_by_store
average_sales_store = average_sales_by_store[average_sales_by_store['store'] == 'Kaggle Store']['average_sales'].values[0]

percentage_drop_store = (lowest_sales_store - average_sales_store) / average_sales_store * 100
"Between the end of 2019 and april 2020, Kaggle Store experienced a drop of {:.2f}% in sales".format(percentage_drop_store)

Observations: 

- Kagglazon sells consistently the most courses, by an order of magnitude more than Kaggle Learn/Store
- The effect of april 2020 dip is seen more pronounced there, while in Kaggle Learn/Store it's much smaller at first glance
- After calculating the relative drop in sales, it is the same -37% for all three stores

### product

In [ ]:
print("Number of unique products: ", df_train['product'].nunique())
total_sales_by_product = df_train[['product', 'num_sold']].groupby('product').sum().reset_index().sort_values(by='num_sold', ascending=False)
total_sales_by_product

In [ ]:
product_palette = dict(zip(total_sales_by_product['product'], sns.color_palette('Set2', len(total_sales_by_product['product']))))

In [ ]:
total_sales_by_product_daily = df_train[['date', 'product', 'num_sold']].groupby(['date', 'product']).sum().reset_index()
total_sales_by_product_daily.columns = ['date', 'product', 'total_sales']

fig, ax = plt.subplots(figsize=(15, 5))
sns.lineplot(x='date', y='total_sales', hue='product', data=total_sales_by_product_daily, ax=ax, palette=product_palette)
ax.set_xlabel('Date')
ax.set_title('Number of Sales by product')

In [ ]:
products = df_train['product'].unique()
# remove "Using LLMs to Win Friends and Influence People" from products
products = products[products != "Using LLMs to Win Friends and Influence People"]

# for each product plot lineplot of total_sales
for product in products:
    product_sales = total_sales_by_product_daily[total_sales_by_product_daily['product'] == product]
    fig, ax = plt.subplots(figsize=(15, 5))
    sns.lineplot(x='date', y='total_sales', data=product_sales, ax=ax, color=product_palette[product])
    ax.set_title(f'Number of Sales for {product}')
    plt.show()

Observations:

- here we clearly see the seasonality and periodicallity of our data (except the light green line)

### Country x product

In [ ]:
df_train[['country', 'product', 'num_sold']]

# create a new column called country_x_product
df_train['country_x_product'] = df_train['country'] + ' x ' + df_train['product']
df_train[['country', 'product', 'country_x_product']]
print("Number of unique country x product: ", df_train['country_x_product'].nunique())

# show number of sales for each country x product
total_sales_by_country_x_product = df_train[['country_x_product', 'num_sold']].groupby('country_x_product').sum().reset_index().sort_values(by='num_sold', ascending=False)
total_sales_by_country_x_product

In [ ]:
top_country_x_product = total_sales_by_country_x_product.head(5)
top_country_x_product = list(top_country_x_product['country_x_product'])


fig, ax = plt.subplots(figsize=(15, 5))

plot_data = df_train[df_train['country_x_product'].isin(top_country_x_product)]
cartesian_palette = sns.color_palette('husl', len(top_country_x_product))

sns.lineplot(x='date', y='num_sold', hue='country_x_product', data=plot_data, ax=ax, palette=cartesian_palette)
ax.set_xlabel('Date')
ax.set_ylabel('Number of Sales')
ax.set_title('Number of Sales by combination of Country x Product')
plt.show()